#**Génération d'Embeddings avec DistilBERT pour la Recherche d'Articles Scientifiques via Chroma**

##**Introduction**

Ce notebook présente un processus complet pour la génération d'embeddings de segments d'articles scientifiques en utilisant DistilBERT, un modèle allégé et performant pour les tâches de Natural Language Processing (NLP). L'objectif est de permettre une recherche sémantique efficace d'articles via une base de données d'embeddings stockés dans ChromaDB.

____________________________

In [ ]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Sujet1NLP3D

/content/drive/MyDrive/Sujet1NLP3D


###**1. Installation et Importation des Bibliothèques Nécessaires**

Dans cette section, nous installons et importons les bibliothèques requises pour traiter le texte, générer les embeddings et interagir avec les API Google pour accéder aux fichiers.

In [ ]:
!pip install gensim transformers sentence-transformers torch chromadb gspread oauth2client spacy tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.



- *gensim et sentence-transformers* : Ces bibliothèques permettent la manipulation des modèles pré-entraînés et des embeddings.
- *transformers* : Nécessaire pour travailler avec le modèle DistilBERT.
- *chromadb* : Utilisé pour le stockage et l'interrogation des embeddings générés.
- *spacy et tiktoken* : Utilisés pour le traitement du texte et la tokenisation.

In [ ]:
import os
import re
import gensim
import torch
import chromadb
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import time
import spacy
from chromadb.utils import embedding_functions
from transformers import DistilBertTokenizer, DistilBertModel

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Initialisation de Spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Indiquer le chemin vers le répertoire de Chroma dans Google Drive
CHROMA_PATH = "chroma_db_persistent"

# Initialiser le client Chroma avec le répertoire persistant
client = chromadb.PersistentClient(path=CHROMA_PATH) # Set the persist directory here

# Create or get the collection, persist_directory is no longer needed.
collection = client.get_or_create_collection("Article_Embedding")

In [ ]:
# Authentification avec l'API Google Drive
creds = ServiceAccountCredentials.from_json_keyfile_name('tools/custom-casing-434908-n5-4d3c56433812.json', ["https://www.googleapis.com/auth/drive"])
drive_service = build('drive', 'v3', credentials=creds)

In [ ]:
# Connectez-vous à Google Sheets avec vos informations d'identification
client = gspread.authorize(creds)

In [ ]:
# Ouvrez votre fichier Excel qui contient les articles
sheet = client.open('arxiv_data').sheet1

In [ ]:
# Récupérer les données de l'Excel
data = sheet.get_all_records()

###**2. Extraction des Articles**

Nous définissons ici les fonctions pour extraire le contenu des articles depuis Google Drive et les segmenter en sections et paragraphes.

In [ ]:
# Fonction pour extraire l'ID du fichier à partir du lien Google Drive
def extract_file_id(drive_link):
    match = re.search(r'/d/([a-zA-Z0-9_-]+)', drive_link)
    return match.group(1) if match else None

In [ ]:
# Fonction pour télécharger le fichier texte depuis Google Drive
def download_file(file_id, file_name):
    request = drive_service.files().get_media(fileId=file_id)
    file_path = f'/content/{file_name}.txt'
    with open(file_path, 'wb') as f:
        downloader = MediaIoBaseDownload(f, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
    return file_path

Fonction extract_file_id : Cette fonction utilise des expressions régulières pour extraire l'ID unique d'un fichier Google Drive à partir d'un lien de partage. Cet ID est ensuite utilisé pour télécharger le fichier.

###**3. Initialisation des Modèles et de la Base de Données Chroma**

Dans cette partie, nous initialisons le modèle DistilBERT ainsi que Chroma, une base de données vectorielle où seront stockés les embeddings générés.

In [ ]:
# Charger les modèles
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

- *DistilBERT* est un modèle pré-entraîné de la famille BERT, mais il est plus léger et rapide, tout en maintenant une bonne performance pour les tâches de NLP.
- *Chroma* est une base de données qui permet de stocker les embeddings de manière efficace pour une recherche rapide et sémantique.

###**4. Génération des Embeddings avec DistilBERT**

Nous utilisons **DistilBERT** pour convertir chaque segment de texte en embeddings numériques, qui capturent le sens et le contexte de chaque segment.

In [ ]:
def get_distilbert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy().reshape(-1)  # Reshape to 1D

- *DistilBERT* prend en entrée des segments de texte (phrases ou paragraphes) et les convertit en vecteurs numériques. Ces vecteurs sont appelés *embeddings*, et ils représentent la signification sémantique du texte.
- *Pourquoi utiliser les embeddings ?* : Les embeddings permettent de mesurer la similarité entre des segments de texte de manière numérique, facilitant ainsi la recherche de contenus similaires.

###**5. Segmentation des Articles**

In [ ]:
# Fonction pour segmenter un article en titres, sections, et paragraphes
def segment_article(content):
    """
    Segmente l'article en titre, sections et paragraphes.
    - Le titre est sur la première ligne.
    - Les sections sont séparées du texte précédent et suivant par 3 sauts de ligne.
    - Les paragraphes sont séparés par 2 sauts de ligne.
    """
    # Séparer le contenu en lignes
    lines = content.splitlines()

    # Extraire le titre (première ligne)
    title = lines[0].strip()

    # Reconstruire le texte sans le titre
    text = "\n".join(lines[1:]).strip()

    # Diviser d'abord en sections en utilisant 3 sauts de ligne
    sections = text.split("\n\n\n")

    segmented_text = []

    # Parcourir chaque section pour identifier les paragraphes
    for section in sections:
        section = section.strip()
        if section:
            # Diviser chaque section en paragraphes avec 2 sauts de ligne
            paragraphs = section.split("\n\n")
            paragraphs = [p.strip() for p in paragraphs if p.strip()]

            # Si la section commence par un titre, l'ajouter en majuscule
            if paragraphs[0].isupper():
                segmented_text.append(paragraphs[0])  # Ajouter la section en majuscule
                paragraphs = paragraphs[1:]  # Retirer le titre de la liste des paragraphes

            # Ajouter les paragraphes
            segmented_text.extend(paragraphs)

    return title, segmented_text

La segmentation est une étape essentielle où chaque article est divisé en sections plus petites (paragraphes). Cela permet d'améliorer la granularité des embeddings, en générant des vecteurs pour chaque segment au lieu d'un seul vecteur pour l'ensemble du document.

- **Pourquoi la segmentation est-elle importante ?** Elle permet une recherche plus précise et contextualisée, en associant chaque paragraphe ou section à une partie spécifique de l'article. Cela améliore la qualité des résultats lors des requêtes sémantiques.

In [ ]:
# Dossier contenant les fichiers segmentés
segmented_dir = 'new_segmented_files'

# Initialiser la liste pour stocker les contenus des fichiers
corpus = []
file_names = []

# Vérifier que le dossier existe et charger les articles
if os.path.exists(segmented_dir):
    for file_name in os.listdir(segmented_dir):
        if file_name.endswith(".txt"):
            file_path = os.path.join(segmented_dir, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                corpus.append(content)  # Ajouter le contenu de l'article à la liste
                file_names.append(file_name)  # Ajouter le nom du fichier
    print(f"{len(corpus)} articles ont été chargés depuis {segmented_dir}.")
else:
    print(f"Le dossier {segmented_dir} n'existe pas.")

200 articles ont été chargés depuis new_segmented_files.


In [ ]:
# Utiliser uniquement DistilBERT pour générer et stocker les embeddings dans Chroma
embedding_func = get_distilbert_embeddings

###**6. Stockage des Embeddings dans ChromaDB**

Une fois les embeddings générés pour chaque segment d'article, nous les stockons dans **ChromaDB** pour qu'ils puissent être facilement interrogés lors des requêtes.

In [ ]:
# Stocker les embeddings pour chaque paragraphe/section
for i, content in enumerate(corpus):
    # Segmenter l'article en titre, sections et paragraphes
    title, segmented_text = segment_article(content)

    # Générer les embeddings pour chaque paragraphe/section
    for j, paragraph in enumerate(segmented_text):
        embeddings = embedding_func(paragraph)

        # Convertir l'embedding en liste
        embeddings = embeddings.tolist()

        # Ajouter les embeddings pour chaque paragraphe dans Chroma
        collection.add(
            ids=[f"{i}_{j}"],  # Utiliser l'index de l'article et du paragraphe
            documents=[paragraph],  # Stocker chaque paragraphe/section
            embeddings=[embeddings],  # Embedding du paragraphe/section
            metadatas=[{"file_name": file_names[i], "paragraph_index": j}]  # Stocker le nom de fichier et l'indice du paragraphe
        )
print(f"Les embeddings pour chaque paragraphe/section ont été stockés dans Chroma à {CHROMA_PATH}.")

Les embeddings pour chaque paragraphe/section ont été stockés dans Chroma à chroma_db_persistent.


- Chaque embedding est associé à des métadonnées qui permettent de savoir de quel fichier et de quelle section de l'article il provient. Cela facilite l'identification des résultats les plus pertinents lors des recherches.
- **ChromaDB** est conçu pour être une base de données optimisée pour le stockage et la recherche vectorielle, ce qui la rend particulièrement adaptée aux embeddings.

###**7. Interrogation de la Base de Données via une Requête**

Une fois les embeddings stockés, nous pouvons interroger la base de données pour trouver les segments d'articles les plus similaires à une requête donnée.

In [ ]:
# Interroger Chroma avec une requête exemple
query = "Deep learning techniques for image segmentation in computer vision"

# Use the same embedding function that was used to generate embeddings for the collection
# Assuming GloVe was the best model and its embedding function is 'get_glove_embeddings':
query_embedding = embedding_func(query)  # Instead of get_sentence_embeddings

# Rechercher les articles les plus similaires dans la collection
results = collection.query(
    query_embeddings=[query_embedding.tolist()],
    n_results=5  # Nombre de résultats à retourner
)

# Afficher les résultats (documents et métadonnées)
for i, result in enumerate(results['documents']):
    print(f"Article {i+1}: {result}")
    print(f"Métadonnées associées : {results['metadatas'][i]}")

Article 1: ['C. LEARNING-ORIENTED COMMUNICATION AND TARGET SENSING', '2.1 A NEURAL DYNAMICS MODEL OF VISUAL DECISION-MAKING WITH', 'E. PREDICTION VALIDATION BY IN-CONTEXT LEARNING.', 'B. OMMER, HIGH-RESOLUTION IMAGE SYNTHESIS WITH', 'PROPOSED ANALYSIS METHODS FOR PREDICTION TASK DATA']
Métadonnées associées : [{'file_name': 'article_137_segmented.txt', 'paragraph_index': 7}, {'file_name': 'article_55_segmented.txt', 'paragraph_index': 4}, {'file_name': 'article_198_segmented.txt', 'paragraph_index': 21}, {'file_name': 'article_103_segmented.txt', 'paragraph_index': 34}, {'file_name': 'article_159_segmented.txt', 'paragraph_index': 36}]


- **Fonctionnement de la recherche** : Une requête textuelle est convertie en embedding, et nous comparons cet embedding aux embeddings stockés dans Chroma pour identifier les segments les plus similaires.
- **Résultats pertinents** : Les résultats incluent non seulement les segments similaires, mais aussi leurs métadonnées, ce qui permet de retrouver rapidement la source de l'information.

##**Conclusion**

Ce notebook offre une approche complète pour le traitement, l'embedding, et la recherche de texte dans des articles scientifiques. L'utilisation de DistilBERT et de ChromaDB permet d'optimiser la recherche sémantique, et le processus peut être facilement adapté à différents cas d'usage, tels que la recherche documentaire ou les systèmes de questions-réponses basés sur le texte.